In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os

%reload_ext autoreload
%autoreload 2

from networks.models import load_model
from torch import cuda
from data.utils import SettingsTraining
from torch.utils.data import DataLoader
import torch
from main import init_data
import logging
from datetime import datetime
from typing import Dict
from math import inf
from torch.utils.data import DataLoader
from networks.unet import UNet
from utils.visualize_data import DataToVisualize, _aligned_colorbar

In [ ]:
def run(settings: SettingsTraining):
    dataset, dataloaders = init_data(settings)
    if settings.device is None:
        settings.device = "cuda" if cuda.is_available() else "cpu"
    # model choice
    in_channels = dataset.input_channels
    if settings.case in ["test", "finetune"]:
        model = load_model({"model_choice": settings.model_choice,
                           "in_channels": in_channels}, settings.path_to_model)
    model.to(settings.device)

    # load model (for application only)
    model = load_model({"model_choice": settings.model_choice, "in_channels": in_channels}, os.path.join(settings.path_to_model), "model", settings.device)
    model.to(settings.device)

    plot_sample(model, dataloaders["test"], settings.device, plot_name=settings.name_folder_destination + "/plot_test_sample", amount_plots=10,)


In [ ]:
def plot_sample(model: UNet, dataloader: DataLoader, device: str, amount_plots: int = inf, plot_name: str = "default"):

    logging.warning("Plotting...")
    error = []
    error_mean = []

    if amount_plots > len(dataloader.dataset):
        amount_plots = len(dataloader.dataset)

    current_id = 0
    norm = dataloader.dataset.dataset.norm
    for inputs, labels in dataloader:
        len_batch = inputs.shape[0]
        for datapoint_id in range(len_batch):
            # get data
            start_time = datetime.now()
            x = inputs[datapoint_id].to(device)
            x = torch.unsqueeze(x, 0)
            model.eval()
            y_out = model(x).to(device)

            # reverse transform for plotting real values
            y = labels[datapoint_id]
            x = norm.reverse(x.detach().cpu().squeeze(), "Inputs")
            y = norm.reverse(y.detach().cpu(),"Labels")[0]
            y_out = norm.reverse(y_out.detach().cpu()[0],"Labels")[0]
            print(f"Index {datapoint_id}")
            print(f"Time of inference: {(datetime.now() - start_time).total_seconds()}s")
            loc_max = y_out.argmax()
            print(f"Max temp: {y_out.max()} at {(loc_max%100, loc_max//100%1280, loc_max//100//1280%5)}")

            # calculate error
            error_current = y-y_out
            error.append(abs(error_current))
            error_mean.append(
                torch.mean(error_current).item())

            # plot temperature true, temperature out, error, physical variables
            temp_max = max(y.max(), y_out.max())
            temp_min = min(y.min(), y_out.min())
            info = dataloader.dataset.dataset.info
            extent_highs = (np.array(info["CellsSize"][:2]) * y.shape)
            dict_to_plot = {
                "t_true": DataToVisualize(y, "Temperature True [°C]",extent_highs, {"vmax": temp_max, "vmin": temp_min}),
                "t_out": DataToVisualize(y_out, "Temperature Out [°C]",extent_highs, {"vmax": temp_max, "vmin": temp_min}),
                "error": DataToVisualize(torch.abs(error_current), "Abs. Error [°C]",extent_highs),
            }
            physical_vars = info["Inputs"].keys()
            for physical_var in physical_vars:
                index = info["Inputs"][physical_var]["index"]
                dict_to_plot[physical_var] = DataToVisualize(
                    x[index], physical_var,extent_highs)

            name_pic = f"runs/{plot_name}_{current_id}"
            figsize_x = extent_highs[0]/extent_highs[1]*3
            _plot_datafields(dict_to_plot, name_pic=name_pic, figsize_x=figsize_x)

            logging.info(f"Resulting pictures are at runs/{plot_name}_*")

            # if (current_id > 0 and current_id % 6 == 0) or current_id >= amount_plots-1:
            #     plt.close("all")

            if current_id >= amount_plots-1:
                max_error = np.max(error[-1].cpu().numpy())
                logging.info("Maximum error: ", max_error)
                return error_mean, max_error
            current_id += 1

            # break


In [ ]:
def _plot_datafields(data: Dict[str, DataToVisualize], name_pic: str, figsize_x: float = 38.4):
    _, axes = plt.subplots(3, 1, sharex=True,
                           figsize=(figsize_x, 3*(3)))

    for index, (name, datapoint) in enumerate(data.items()):
        plt.sca(axes[index])
        if name in ["t_true", "t_out", "error"]:  
            if name=="error": datapoint.contourargs["levels"] = [level - 10.6 for level in datapoint.contourargs["levels"]] 
            CS = plt.contour(torch.flip(datapoint.data, dims=[1]).T, **datapoint.contourargs)
            plt.clabel(CS, inline=1, fontsize=10)
            if index==1:
                path = CS.collections[0].get_paths()
                print(path[0])
        if index >= 2: break

In [ ]:
args = {
"datasets_path" : "datasets_prepared",
"device" : "cuda:3",
"epochs" : 1,
"model_choice" : "unet",
"dataset_name" : "benchmark_testcases_4_assumedsteadystate",
"case" : "test",
"path_to_model" : "current_unet_benchmark_dataset_2d_100datapoints_assumedsteadystate",
}
args["name_folder_destination"] = f'current_{args["model_choice"]}_{args["dataset_name"]}'

settings = SettingsTraining(**args)
run(settings)

# Testcase 0

Time of inference: 0.00682s

Max temp: 16.320730209350586 at (tensor(43), tensor(0), tensor(0))

In [ ]:
points = np.array([
       [270.33353481,  80.        ],
       [271.05882353,  78.86741076],
       [273.93532175,  74.66666667],
       [276.07843137,  71.44656249],
       [277.18519894,  69.33333333],
       [281.09803922,  64.72622515],
       [281.73581464,  64.        ],
       [283.63455035,  58.66666667],
       [285.00140942,  53.33333333],
       [286.11764706,  50.68294023],
       [287.21191799,  48.63189567]])

In [ ]:
# get max and min of points[:,0] and points[:,1]
x_min = points[:,0].min()
x_max = points[:,0].max()
y_min = points[:,1].min()
y_max = points[:,1].max()
width = y_max - y_min
length = x_max - x_min
print(f"width = {width}, length = {length}")
print(f"x_max = {x_max}, x_min = {x_min}")
print(f"y_max = {y_max}, y_min = {y_min}")

# Testcase 1

Time of inference: 0.003576s

Max temp: 15.559391021728516 at (tensor(39), tensor(4), tensor(0))

In [ ]:
points = np.array([
       [1280.        ,   17.05727315],
       [1277.87285337,   16.        ],
       [1274.98039216,   14.74106599],
       [1269.96078431,   14.23360169],
       [1264.94117647,   13.7492677 ],
       [1259.92156863,   13.84534452],
       [1254.90196078,   14.02215654],
       [1249.88235294,   14.39152419],
       [1244.8627451 ,   14.60922755],
       [1239.84313725,   15.1001274 ],
       [1234.82352941,   15.06990318],
       [1229.80392157,   14.92917572],
       [1224.78431373,   14.78642901],
       [1219.76470588,   14.89445626],
       [1214.74509804,   14.6969652 ],
       [1209.7254902 ,   15.09665306],
       [1204.70588235,   14.76644232],
       [1199.68627451,   15.12197435],
       [1194.66666667,   14.80020802],
       [1189.64705882,   15.10505504],
       [1184.62745098,   14.82100102],
       [1179.60784314,   15.21580036],
       [1174.58823529,   14.93967968],
       [1169.56862745,   15.36467925],
       [1164.54901961,   15.06983254],
       [1159.52941176,   15.42441896],
       [1154.50980392,   15.09578299],
       [1149.49019608,   15.46087576],
       [1144.47058824,   15.12679631],
       [1139.45098039,   15.45187427],
       [1134.43137255,   15.14373209],
       [1129.41176471,   15.48432569],
       [1124.39215686,   15.17258139],
       [1119.37254902,   15.47505302],
       [1114.35294118,   15.17164889],
       [1109.33333333,   15.50340421],
       [1104.31372549,   15.20520872],
       [1099.29411765,   15.47119428],
       [1094.2745098 ,   15.17188128],
       [1089.25490196,   15.43976994],
       [1084.23529412,   15.15327632],
       [1079.21568627,   15.34552269],
       [1074.19607843,   15.06747273],
       [1069.17647059,   15.24599933],
       [1064.15686275,   14.99604139],
       [1059.1372549 ,   15.08797847],
       [1054.11764706,   14.85449637],
       [1049.09803922,   14.91753113],
       [1044.07843137,   14.71408263],
       [1039.05882353,   14.70340825],
       [1034.03921569,   14.53871877],
       [1029.01960784,   14.48565151],
       [1024.        ,   14.35460412],
       [1018.98039216,   14.26199133],
       [1013.96078431,   14.15806997],
       [1008.94117647,   14.0458638 ],
       [1003.92156863,   13.96293754],
       [ 998.90196078,   13.79828595],
       [ 993.88235294,   13.73428408],
       [ 988.8627451 ,   13.55365429],
       [ 983.84313725,   13.51266511],
       [ 978.82352941,   13.2992076 ],
       [ 973.80392157,   13.29200855],
       [ 968.78431373,   13.07817427],
       [ 963.76470588,   13.11061176],
       [ 958.74509804,   12.88235468],
       [ 953.7254902 ,   12.95473329],
       [ 948.70588235,   12.72750558],
       [ 943.68627451,   12.82635167],
       [ 938.66666667,   12.59156198],
       [ 933.64705882,   12.70581492],
       [ 928.62745098,   12.47813713],
       [ 923.60784314,   12.59696147],
       [ 918.58823529,   12.35281493],
       [ 913.56862745,   12.47335098],
       [ 908.54901961,   12.23145128],
       [ 903.52941176,   12.36358957],
       [ 898.50980392,   12.12973001],
       [ 893.49019608,   12.28309225],
       [ 888.47058824,   12.09077001],
       [ 883.45098039,   12.25018258],
       [ 878.43137255,   12.06351718],
       [ 873.41176471,   12.22270711],
       [ 868.39215686,   12.06080147],
       [ 863.37254902,   12.21887737],
       [ 858.35294118,   12.04718788],
       [ 853.33333333,   12.2124142 ],
       [ 848.31372549,   12.03662296],
       [ 843.29411765,   12.21519401],
       [ 838.2745098 ,   12.03435627],
       [ 833.25490196,   12.23861048],
       [ 828.23529412,   12.10596427],
       [ 823.21568627,   12.32959363],
       [ 818.19607843,   12.19982043],
       [ 813.17647059,   12.424869  ],
       [ 808.15686275,   12.31760909],
       [ 803.1372549 ,   12.51481323],
       [ 798.11764706,   12.36384622],
       [ 793.09803922,   12.51637808],
       [ 788.07843137,   12.34742423],
       [ 783.05882353,   12.45290742],
       [ 778.03921569,   12.21724144],
       [ 773.01960784,   12.27867291],
       [ 768.        ,   12.03648258],
       [ 762.98039216,   12.10142489],
       [ 757.96078431,   11.83640527],
       [ 752.94117647,   11.89033918],
       [ 747.92156863,   11.67991993],
       [ 742.90196078,   11.79353795],
       [ 737.88235294,   11.60305603],
       [ 732.8627451 ,   11.75175625],
       [ 727.84313725,   11.65036685],
       [ 722.82352941,   11.89907578],
       [ 717.80392157,   11.8244767 ],
       [ 712.78431373,   12.08165335],
       [ 707.76470588,   12.04958852],
       [ 702.74509804,   12.35029858],
       [ 697.7254902 ,   12.28839903],
       [ 692.70588235,   12.59167288],
       [ 687.68627451,   12.57445099],
       [ 682.66666667,   12.91665224],
       [ 677.64705882,   12.89254039],
       [ 672.62745098,   13.22526756],
       [ 667.60784314,   13.25779724],
       [ 662.58823529,   13.5979003 ],
       [ 657.56862745,   13.6375494 ],
       [ 652.54901961,   13.94728379],
       [ 647.52941176,   14.03067426],
       [ 642.50980392,   14.30825651],
       [ 637.49019608,   14.3663363 ],
       [ 632.47058824,   14.59365253],
       [ 627.45098039,   14.69298638],
       [ 622.43137255,   14.8495138 ],
       [ 617.41176471,   14.88695775],
       [ 612.39215686,   14.99555413],
       [ 607.37254902,   15.05584722],
       [ 602.35294118,   15.08719465],
       [ 597.33333333,   15.07725661],
       [ 592.31372549,   15.12755013],
       [ 587.29411765,   15.17418425],
       [ 582.2745098 ,   15.20604519],
       [ 577.25490196,   15.22636859],
       [ 572.23529412,   15.30366532],
       [ 567.21568627,   15.36819293],
       [ 562.19607843,   15.43233773],
       [ 557.17647059,   15.44085459],
       [ 552.15686275,   15.53645217],
       [ 547.1372549 ,   15.56724529],
       [ 542.11764706,   15.62775911],
       [ 537.09803922,   15.58373386],
       [ 532.07843137,   15.65748244],
       [ 527.05882353,   15.64539105],
       [ 522.03921569,   15.69356715],
       [ 517.01960784,   15.63415925],
       [ 512.        ,   15.72280994],
       [ 506.98039216,   15.7045471 ],
       [ 501.96078431,   15.7531067 ],
       [ 496.94117647,   15.70093805],
       [ 491.92156863,   15.77827178],
       [ 486.90196078,   15.7831433 ],
       [ 481.88235294,   15.88475564],
       [ 476.8627451 ,   15.85918539],
       [ 471.85829809,   16.        ],
       [ 471.84313725,   16.00031133],
       [ 466.82352941,   16.01757975],
       [ 461.80392157,   16.11291937],
       [ 456.78431373,   16.08446579],
       [ 451.76470588,   16.21691572],
       [ 446.74509804,   16.19223263],
       [ 441.7254902 ,   16.33376478],
       [ 436.70588235,   16.27959196],
       [ 431.68627451,   16.44983493],
       [ 426.66666667,   16.41936376],
       [ 421.64705882,   16.5710041 ],
       [ 416.62745098,   16.5199577 ],
       [ 411.60784314,   16.68631306],
       [ 406.58823529,   16.65955906],
       [ 401.56862745,   16.81743759],
       [ 396.54901961,   16.78726252],
       [ 391.52941176,   16.94276028],
       [ 386.50980392,   16.94360815],
       [ 381.49019608,   17.08259916],
       [ 376.47058824,   17.08459976],
       [ 371.45098039,   17.22507658],
       [ 366.43137255,   17.26466275],
       [ 361.41176471,   17.41422564],
       [ 356.39215686,   17.43825721],
       [ 351.37254902,   17.60230348],
       [ 346.35294118,   17.64813645],
       [ 341.33333333,   17.81318033],
       [ 336.31372549,   17.8215663 ],
       [ 331.29411765,   18.01210625],
       [ 326.2745098 ,   18.00931426],
       [ 321.25490196,   18.21316126],
       [ 316.23529412,   18.19621989],
       [ 311.21568627,   18.4180341 ],
       [ 306.19607843,   18.39090375],
       [ 301.17647059,   18.60610517],
       [ 296.15686275,   18.54991144],
       [ 291.1372549 ,   18.77183389],
       [ 286.11764706,   18.71560911],
       [ 281.09803922,   18.91805172],
       [ 276.07843137,   18.82561679],
       [ 271.05882353,   19.07675261],
       [ 266.03921569,   18.98744407],
       [ 261.01960784,   19.22328736],
       [ 256.        ,   19.09826293],
       [ 250.98039216,   19.40760768],
       [ 245.96078431,   19.26846179],
       [ 240.94117647,   19.57423711],
       [ 235.92156863,   19.40431658],
       [ 230.90196078,   19.78586029],
       [ 225.88235294,   19.63735904],
       [ 220.8627451 ,   19.98853796],
       [ 215.84313725,   19.89243937],
       [ 210.82352941,   20.28921709],
       [ 205.80392157,   20.20374996],
       [ 200.78431373,   20.55697656],
       [ 195.76470588,   20.4822496 ],
       [ 190.74509804,   20.93027703],
       [ 185.7254902 ,   20.83147992],
       [ 180.70588235,   21.20301352],
       [ 175.68627451,   21.12412561],
       [ 173.39834683,   21.33333333],
       [ 170.66666667,   21.46461804],
       [ 165.64705882,   21.48719969],
       [ 160.62745098,   21.71910475],
       [ 155.60784314,   21.81074645],
       [ 150.58823529,   21.92556062],
       [ 145.56862745,   22.24039978],
       [ 140.54901961,   22.09513847],
       [ 135.52941176,   22.18458371],
       [ 130.50980392,   22.30610114],
       [ 125.49019608,   23.48141904],
       [ 120.47058824,   25.22185902],
       [ 117.54292031,   26.66666667],
       [ 115.45098039,   27.32493727],
       [ 111.41885864,   32.        ],
       [ 110.43137255,   34.23088715],
       [ 108.60534307,   37.33333333],
       [ 110.43137255,   40.62081707],
       [ 111.24877141,   42.66666667],
       [ 115.45098039,   47.39852888],
       [ 116.83885013,   48.        ],
       [ 120.47058824,   50.14028649],
       [ 125.49019608,   51.81441911],
       [ 130.50980392,   52.49390256],
       [ 135.52941176,   52.6592197 ],
       [ 140.54901961,   52.80432314],
       [ 145.56862745,   52.98713696],
       [ 148.90067781,   53.33333333],
       [ 150.58823529,   53.62095772],
       [ 155.60784314,   53.78116474],
       [ 160.62745098,   54.11808441],
       [ 165.64705882,   54.36477436],
       [ 170.66666667,   54.44944407],
       [ 175.68627451,   54.58989127],
       [ 180.70588235,   54.73641713],
       [ 185.7254902 ,   54.82226577],
       [ 190.74509804,   54.9517568 ],
       [ 195.76470588,   55.06999558],
       [ 200.78431373,   55.2585516 ],
       [ 205.80392157,   55.34691157],
       [ 210.82352941,   55.45995689],
       [ 215.84313725,   55.57064154],
       [ 220.8627451 ,   55.65874493],
       [ 225.88235294,   55.72065697],
       [ 230.90196078,   55.74064038],
       [ 235.92156863,   55.8045726 ],
       [ 240.94117647,   55.85385442],
       [ 245.96078431,   55.92172767],
       [ 250.98039216,   55.93098506],
       [ 256.        ,   56.01566723],
       [ 261.01960784,   56.07007782],
       [ 266.03921569,   56.15995458],
       [ 271.05882353,   56.17689115],
       [ 276.07843137,   56.26601811],
       [ 281.09803922,   56.31654429],
       [ 286.11764706,   56.4090556 ],
       [ 291.1372549 ,   56.42254113],
       [ 296.15686275,   56.51641957],
       [ 301.17647059,   56.55716104],
       [ 306.19607843,   56.67539508],
       [ 311.21568627,   56.70913306],
       [ 316.23529412,   56.82508071],
       [ 321.25490196,   56.86604481],
       [ 326.2745098 ,   57.02677748],
       [ 331.29411765,   57.04992213],
       [ 336.31372549,   57.19051042],
       [ 341.33333333,   57.21031055],
       [ 346.35294118,   57.38708915],
       [ 351.37254902,   57.39514249],
       [ 356.39215686,   57.53269615],
       [ 361.41176471,   57.54419806],
       [ 366.43137255,   57.72717682],
       [ 371.45098039,   57.7300077 ],
       [ 376.47058824,   57.88987863],
       [ 381.49019608,   57.88508868],
       [ 386.50980392,   58.07370821],
       [ 391.52941176,   58.074328  ],
       [ 396.54901961,   58.24731966],
       [ 401.56862745,   58.24058783],
       [ 406.58823529,   58.43288319],
       [ 411.60784314,   58.45305135],
       [ 416.62745098,   58.61206746],
       [ 421.64705882,   58.6431759 ],
       [ 422.35682865,   58.66666667],
       [ 426.66666667,   58.88977384],
       [ 431.68627451,   58.93491529],
       [ 436.70588235,   59.0744786 ],
       [ 441.7254902 ,   59.12306874],
       [ 446.74509804,   59.26980598],
       [ 451.76470588,   59.29437425],
       [ 456.78431373,   59.3771185 ],
       [ 461.80392157,   59.41002094],
       [ 466.82352941,   59.49064387],
       [ 471.84313725,   59.51375703],
       [ 476.8627451 ,   59.5501258 ],
       [ 481.88235294,   59.60344527],
       [ 486.90196078,   59.64005211],
       [ 491.92156863,   59.69021153],
       [ 496.94117647,   59.6834789 ],
       [ 501.96078431,   59.74993278],
       [ 506.98039216,   59.74478553],
       [ 512.        ,   59.78458252],
       [ 517.01960784,   59.77739029],
       [ 522.03921569,   59.82780429],
       [ 527.05882353,   59.81830915],
       [ 532.07843137,   59.84746984],
       [ 537.09803922,   59.8539937 ],
       [ 542.11764706,   59.88950306],
       [ 547.1372549 ,   59.91000528],
       [ 552.15686275,   59.92157894],
       [ 557.17647059,   59.94317473],
       [ 562.19607843,   59.96157231],
       [ 567.21568627,   59.97273149],
       [ 572.23529412,   59.99741672],
       [ 577.25490196,   60.01546162],
       [ 582.2745098 ,   60.03799366],
       [ 587.29411765,   60.03272933],
       [ 592.31372549,   60.05672725],
       [ 597.33333333,   60.07362454],
       [ 602.35294118,   60.10463455],
       [ 607.37254902,   60.10599986],
       [ 612.39215686,   60.14813607],
       [ 617.41176471,   60.18293187],
       [ 622.43137255,   60.24543147],
       [ 627.45098039,   60.27300259],
       [ 632.47058824,   60.34505667],
       [ 637.49019608,   60.39863281],
       [ 642.50980392,   60.48813482],
       [ 647.52941176,   60.51451875],
       [ 652.54901961,   60.59449073],
       [ 657.56862745,   60.62733348],
       [ 662.58823529,   60.71622599],
       [ 667.60784314,   60.7212173 ],
       [ 672.62745098,   60.79938925],
       [ 677.64705882,   60.81017229],
       [ 682.66666667,   60.90605789],
       [ 687.68627451,   60.91146526],
       [ 692.70588235,   61.00025904],
       [ 697.7254902 ,   61.01063763],
       [ 702.74509804,   61.11457626],
       [ 707.76470588,   61.11356954],
       [ 712.78431373,   61.19420951],
       [ 717.80392157,   61.21203698],
       [ 722.82352941,   61.33773694],
       [ 727.84313725,   61.38491479],
       [ 732.8627451 ,   61.47078707],
       [ 737.88235294,   61.49474372],
       [ 742.90196078,   61.54562107],
       [ 747.92156863,   61.52217573],
       [ 752.94117647,   61.49162198],
       [ 757.96078431,   61.42508105],
       [ 762.98039216,   61.36916477],
       [ 768.        ,   61.27009858],
       [ 773.01960784,   61.16657969],
       [ 778.03921569,   61.07495998],
       [ 783.05882353,   60.98851543],
       [ 788.07843137,   60.90795931],
       [ 793.09803922,   60.82735202],
       [ 798.11764706,   60.77882767],
       [ 803.1372549 ,   60.73297817],
       [ 808.15686275,   60.73045089],
       [ 813.17647059,   60.69075598],
       [ 818.19607843,   60.73212409],
       [ 823.21568627,   60.71595347],
       [ 828.23529412,   60.78178344],
       [ 833.25490196,   60.75165984],
       [ 838.2745098 ,   60.82389886],
       [ 843.29411765,   60.80442922],
       [ 848.31372549,   60.89366388],
       [ 853.33333333,   60.8616133 ],
       [ 858.35294118,   60.96929314],
       [ 863.37254902,   60.93900679],
       [ 868.39215686,   61.0610135 ],
       [ 873.41176471,   61.02061359],
       [ 878.43137255,   61.12723563],
       [ 883.45098039,   61.06176135],
       [ 888.47058824,   61.18038906],
       [ 893.49019608,   61.10433059],
       [ 898.50980392,   61.20072798],
       [ 903.52941176,   61.10804819],
       [ 908.54901961,   61.21854492],
       [ 913.56862745,   61.11635355],
       [ 918.58823529,   61.19289675],
       [ 923.60784314,   61.08273297],
       [ 928.62745098,   61.16008284],
       [ 933.64705882,   61.04044875],
       [ 938.66666667,   61.10169178],
       [ 943.68627451,   60.99075467],
       [ 948.70588235,   61.07632534],
       [ 953.7254902 ,   60.97309128],
       [ 958.74509804,   61.04703572],
       [ 963.76470588,   60.95926451],
       [ 968.78431373,   61.05378802],
       [ 973.80392157,   60.98356017],
       [ 978.82352941,   61.0596523 ],
       [ 983.84313725,   60.99435963],
       [ 988.8627451 ,   61.06602423],
       [ 993.88235294,   61.00870954],
       [ 998.90196078,   61.07434248],
       [1003.92156863,   61.00414063],
       [1008.65856758,   61.07539782]])

In [ ]:
# get max and min of points[:,0] and points[:,1]
x_min = points[:,0].min()
x_max = points[:,0].max()
y_min = points[:,1].min()
y_max = points[:,1].max()
width = y_max - y_min
length = x_max - x_min
print(f"width = {width}, length = {length}")
print(f"x_max = {x_max}, x_min = {x_min}")
print(f"y_max = {y_max}, y_min = {y_min}")

# Testcase 2

Time of inference: 0.006741s

Max temp: 12.426977157592773 at (tensor(71), tensor(4), tensor(0))

In [ ]:
points = np.array([
        [391.24621649,  35.50383687],
       [386.50980392,  35.47073861],
       [381.49019608,  35.45953522],
       [376.47058824,  35.42524795],
       [371.45098039,  35.41266151],
       [366.43137255,  35.3883383 ],
       [361.41176471,  35.36953151],
       [356.39215686,  35.34369447],
       [351.37254902,  35.32063856],
       [346.35294118,  35.30218001],
       [341.33333333,  35.27938354],
       [336.31372549,  35.25835895],
       [331.29411765,  35.23942213],
       [326.2745098 ,  35.23016864],
       [321.25490196,  35.21722108],
       [316.23529412,  35.20798774],
       [311.21568627,  35.21008716],
       [306.19607843,  35.2122718 ],
       [301.17647059,  35.21839497],
       [296.15686275,  35.21234624],
       [291.1372549 ,  35.2254567 ],
       [286.11764706,  35.223986  ],
       [281.09803922,  35.23783298],
       [276.07843137,  35.21869142],
       [271.05882353,  35.24207054],
       [266.03921569,  35.2198592 ],
       [261.01960784,  35.22980606],
       [256.        ,  35.18769328],
       [250.98039216,  35.20914632],
       [245.96078431,  35.16024524],
       [240.94117647,  35.16912501],
       [235.92156863,  35.09983924],
       [230.90196078,  35.13357554],
       [225.88235294,  35.06366845],
       [220.8627451 ,  35.07108923],
       [215.84313725,  35.00191148],
       [210.82352941,  35.02363803],
       [205.80392157,  34.93821983],
       [200.78431373,  34.92055672],
       [195.76470588,  34.8378335 ],
       [190.74509804,  34.8145799 ],
       [185.7254902 ,  34.73498485],
       [180.70588235,  34.70882497],
       [175.68627451,  34.64557488],
       [170.66666667,  34.6282579 ],
       [165.64705882,  34.55614621],
       [160.62745098,  34.53623531],
       [155.60784314,  34.50248839],
       [150.58823529,  34.49072094],
       [145.56862745,  34.35482179],
       [140.54901961,  34.5268386 ],
       [135.52941176,  34.40818157],
       [130.50980392,  34.62919402],
       [125.49019608,  34.59809784],
       [120.47058824,  35.03578811],
       [115.45098039,  35.42375201],
       [112.97658935,  37.33333333],
       [115.45098039,  39.22974901],
       [120.47058824,  39.52094626],
       [125.49019608,  40.03488528],
       [130.50980392,  40.00423546],
       [135.52941176,  40.32709332],
       [140.54901961,  40.30319967],
       [145.56862745,  40.46556376],
       [150.58823529,  40.2948607 ],
       [155.60784314,  40.26608466],
       [160.62745098,  40.19216002],
       [165.64705882,  40.1879027 ],
       [170.66666667,  40.05732068],
       [175.68627451,  40.07392053],
       [180.70588235,  39.9658142 ],
       [185.7254902 ,  39.96532095],
       [190.74509804,  39.86041537],
       [195.76470588,  39.8604494 ],
       [200.78431373,  39.76386446],
       [205.80392157,  39.75687392],
       [210.82352941,  39.66810486],
       [215.84313725,  39.69940809],
       [220.8627451 ,  39.62157713],
       [225.88235294,  39.63679382],
       [230.90196078,  39.56352037],
       [235.92156863,  39.60092541],
       [240.94117647,  39.53183206],
       [245.96078431,  39.54358086],
       [250.98039216,  39.49442887],
       [256.        ,  39.51601787],
       [261.01960784,  39.4815761 ],
       [266.03921569,  39.48934951],
       [271.05882353,  39.47066634],
       [276.07843137,  39.48814634],
       [281.09803922,  39.47716873],
       [286.11764706,  39.48716728],
       [291.1372549 ,  39.48829604],
       [296.15686275,  39.5012872 ],
       [301.17647059,  39.50175096],
       [306.19607843,  39.50482524],
       [311.21568627,  39.50944754],
       [316.23529412,  39.51054911],
       [321.25490196,  39.51140569],
       [326.2745098 ,  39.49742071],
       [331.29411765,  39.49610226],
       [336.31372549,  39.47950634],
       [341.33333333,  39.47136614],
       [346.35294118,  39.44633327],
       [351.37254902,  39.43664043],
       [356.39215686,  39.41470323],
       [361.41176471,  39.40091286],
       [366.43137255,  39.37696659],
       [371.45098039,  39.36158548],
       [376.47058824,  39.34584921],
       [381.49019608,  39.3222654 ],
       [386.50980392,  39.30341755],
       [391.52941176,  39.27723483],
       [396.54901961,  39.26144985],
       [401.56862745,  39.2270923 ],
       [406.58823529,  39.20908589],
       [411.60784314,  39.16221222],
       [416.62745098,  39.14645101],
       [421.64705882,  39.09701256],
       [426.66666667,  39.07289102],
       [431.68627451,  39.01142133],
       [436.70588235,  38.99045362],
       [441.7254902 ,  38.93299641],
       [446.74509804,  38.91004737],
       [451.76470588,  38.86026811],
       [456.78431373,  38.85767731],
       [461.80392157,  38.83090352],
       [466.82352941,  38.84399537],
       [471.84313725,  38.83420716],
       [476.8627451 ,  38.86350917],
       [481.88235294,  38.87504477],
       [486.90196078,  38.8990659 ],
       [491.92156863,  38.9045493 ],
       [496.94117647,  38.92227798],
       [501.96078431,  38.93899911],
       [506.98039216,  38.92688554],
       [512.        ,  38.93627434],
       [517.01960784,  38.89885025],
       [522.03921569,  38.90644598],
       [527.05882353,  38.84581874],
       [532.07843137,  38.85177676],
       [537.09803922,  38.799033  ],
       [542.11764706,  38.83114217],
       [547.1372549 ,  38.78030594],
       [552.15686275,  38.81854984],
       [557.17647059,  38.77567973],
       [562.19607843,  38.83907631],
       [567.21568627,  38.77713106],
       [572.23529412,  38.83748148],
       [577.25490196,  38.76294925],
       [582.2745098 ,  38.82822933],
       [587.29411765,  38.74294027],
       [592.31372549,  38.7932904 ],
       [597.33333333,  38.71338521],
       [602.35294118,  38.79249778],
       [607.37254902,  38.71922796],
       [612.39215686,  38.80221512],
       [617.41176471,  38.74160027],
       [622.43137255,  38.82881304],
       [627.45098039,  38.76547604],
       [632.47058824,  38.84094162],
       [637.49019608,  38.77712856],
       [642.50980392,  38.84981779],
       [647.52941176,  38.77564208],
       [652.54901961,  38.84050932],
       [657.56862745,  38.7611008 ],
       [662.58823529,  38.80316445],
       [667.60784314,  38.71126038],
       [672.62745098,  38.72538496],
       [677.64705882,  38.62497828],
       [682.66666667,  38.62783195],
       [687.68627451,  38.53259247],
       [692.70588235,  38.52065056],
       [697.7254902 ,  38.42735847],
       [702.74509804,  38.41881619],
       [707.76470588,  38.33542286],
       [712.78431373,  38.32282607],
       [717.80392157,  38.25836231],
       [722.82352941,  38.26772733],
       [727.84313725,  38.20674436],
       [732.8627451 ,  38.2151601 ],
       [737.88235294,  38.12300039],
       [742.90196078,  38.15355027],
       [747.92156863,  38.06891815],
       [752.94117647,  38.08692419],
       [757.96078431,  38.02593023],
       [762.98039216,  38.06878847],
       [768.        ,  38.02829539],
       [773.01960784,  38.04295169],
       [778.03921569,  37.99894026],
       [783.05882353,  38.03611398],
       [788.07843137,  37.99801251],
       [793.09803922,  38.02759785],
       [798.11764706,  37.98460868],
       [803.1372549 ,  38.02856852],
       [808.15686275,  37.98983003],
       [813.17647059,  38.0305707 ],
       [818.19607843,  38.00628896],
       [823.21568627,  38.06856871],
       [828.23529412,  38.0576762 ],
       [833.25490196,  38.10962018],
       [838.2745098 ,  38.10805886],
       [843.29411765,  38.14366129],
       [848.31372549,  38.14227638],
       [853.33333333,  38.13661857],
       [858.35294118,  38.12580718],
       [863.37254902,  38.12025271],
       [868.39215686,  38.10500794],
       [873.41176471,  38.07771549],
       [878.43137255,  38.05066988],
       [883.45098039,  38.01392221],
       [888.47058824,  37.97323099],
       [893.49019608,  37.9017548 ],
       [898.50980392,  37.85813469],
       [903.52941176,  37.79796534],
       [908.54901961,  37.75685936],
       [913.56862745,  37.687094  ],
       [918.58823529,  37.63164465],
       [923.60784314,  37.60012455],
       [928.62745098,  37.52379691],
       [933.64705882,  37.46791753],
       [938.66666667,  37.39131676],
       [943.68627451,  37.38267145],
       [946.81699953,  37.33333333],
       [943.68627451,  37.28472886],
       [938.66666667,  37.2714772 ],
       [933.64705882,  37.20064222],
       [928.62745098,  37.13015308],
       [923.60784314,  37.06965792],
       [918.58823529,  37.01632175],
       [913.56862745,  36.98206516],
       [908.54901961,  36.88177903],
       [903.52941176,  36.8694529 ],
       [898.50980392,  36.77190057],
       [893.49019608,  36.76056646],
       [888.47058824,  36.64216697],
       [883.45098039,  36.64239748],
       [878.43137255,  36.55987334],
       [873.41176471,  36.56975525],
       [868.39215686,  36.49636384],
       [863.37254902,  36.52313226],
       [858.35294118,  36.47842769],
       [853.33333333,  36.50610467],
       [848.31372549,  36.46093932],
       [843.29411765,  36.50038246],
       [838.2745098 ,  36.50606499],
       [833.25490196,  36.53804118],
       [828.23529412,  36.56006045],
       [823.21568627,  36.57365324],
       [818.19607843,  36.61269902],
       [813.17647059,  36.60826161],
       [808.15686275,  36.62574008],
       [803.1372549 ,  36.60596141],
       [798.11764706,  36.63177963],
       [793.09803922,  36.60505771],
       [788.07843137,  36.61895895],
       [783.05882353,  36.5967137 ],
       [778.03921569,  36.62277333],
       [773.01960784,  36.59339954],
       [768.        ,  36.59586043],
       [762.98039216,  36.56582715],
       [757.96078431,  36.60372404],
       [752.94117647,  36.54893048],
       [747.92156863,  36.56123567],
       [742.90196078,  36.48151625],
       [737.88235294,  36.50645119],
       [732.8627451 ,  36.42524308],
       [727.84313725,  36.41946481],
       [722.82352941,  36.3736838 ],
       [717.80392157,  36.36523472],
       [712.78431373,  36.31648424],
       [707.76470588,  36.27740922],
       [702.74509804,  36.21485417],
       [697.7254902 ,  36.17587417],
       [692.70588235,  36.10976014],
       [687.68627451,  36.06035273],
       [682.66666667,  35.99818904],
       [677.64705882,  35.95860706],
       [672.62745098,  35.89906304],
       [667.60784314,  35.86184314],
       [662.58823529,  35.82135347],
       [657.56862745,  35.80922977],
       [652.54901961,  35.7888621 ],
       [647.52941176,  35.79643921],
       [642.50980392,  35.78506008],
       [637.49019608,  35.7961377 ],
       [632.47058824,  35.8055752 ],
       [627.45098039,  35.81674786],
       [622.43137255,  35.82980955],
       [617.41176471,  35.84305435],
       [612.39215686,  35.86118734],
       [607.37254902,  35.8749888 ],
       [602.35294118,  35.8747316 ],
       [597.33333333,  35.88573944],
       [592.31372549,  35.88098186],
       [587.29411765,  35.86506978],
       [582.2745098 ,  35.85618009],
       [577.25490196,  35.85216205],
       [572.23529412,  35.85422598],
       [567.21568627,  35.84841613],
       [562.19607843,  35.85668929],
       [557.17647059,  35.85461699],
       [552.15686275,  35.87792193],
       [547.1372549 ,  35.86016465],
       [542.11764706,  35.87228616],
       [537.09803922,  35.84728408],
       [532.07843137,  35.85562697],
       [527.05882353,  35.80882407],
       [522.03921569,  35.80340535],
       [517.01960784,  35.76404279],
       [512.        ,  35.77665559],
       [506.98039216,  35.74972566],
       [501.96078431,  35.78354313],
       [496.94117647,  35.76896382],
       [491.92156863,  35.82665064],
       [486.90196078,  35.80356471],
       [481.88235294,  35.86612299],
       [476.8627451 ,  35.84813963],
       [471.84313725,  35.90994732],
       [466.82352941,  35.87553953],
       [461.80392157,  35.91802323],
       [456.78431373,  35.86893735],
       [451.76470588,  35.89015983],
       [446.74509804,  35.8249873 ],
       [441.7254902 ,  35.8244957 ],
       [436.70588235,  35.75678858],
       [431.68627451,  35.75285191],
       [426.66666667,  35.68813858],
       [421.64705882,  35.68130901],
       [416.62745098,  35.62315831],
       [411.89099983,  35.61966581]])


In [ ]:
# get max and min of points[:,0] and points[:,1]
x_min = points[:,0].min()
x_max = points[:,0].max()
y_min = points[:,1].min()
y_max = points[:,1].max()
width = y_max - y_min
length = x_max - x_min
print(f"width = {width}, length = {length}")
print(f"x_max = {x_max}, x_min = {x_min}")
print(f"y_max = {y_max}, y_min = {y_min}")

# Testcase 3

Time of inference: 0.006819s

Max temp: 11.07814884185791 at (tensor(19), tensor(5), tensor(0))

Path(array([[1280.,    0.]]), None)

In [ ]:
widht = 0
length = 0